In [47]:
import heapq as h
import numpy as np

POISSON_PROCESS_NEW_REQUESTS_LAMBDA = 1
PARETO_PROCESS_NEW_FILE_SIZE_A = 1
PARETO_PROCESS_FILE_POPULARITY_A = 1
LOGNORMAL_PROCESS_ARRIVE_AT_QUEUE_MEAN = 0
LOGNORMAL_PROCESS_ARRIVE_AT_QUEUE_SIGMA = 1

TOTAL_NO_OF_FILES = 10000
INSTITUTIONAL_BANDWIDTH = 100
ACCESS_LINK_BANDWIDTH = 15
TOTAL_TIME_TO_RUN = 10
CACHE_CAPACITY = 1000


class Files:
    def __init__(self):
        self.popularity = np.random.pareto(PARETO_PROCESS_FILE_POPULARITY_A,size=TOTAL_NO_OF_FILES)
        self.popularity = self.popularity/np.sum(self.popularity)
        self.size = np.random.pareto(PARETO_PROCESS_NEW_FILE_SIZE_A,size=TOTAL_NO_OF_FILES)



class Simulation_Q():
    def __init__(self):
        self.q = []
    
    def push(self, event_tuple:tuple):
        h.heappush(self.q,event_tuple)

    def pop(self):
        return h.heappop(self.q)[1]
        
class Cache():
    def __init__(self,
    all_files:Files,
    init_file_list=[]):

        self.capacity = CACHE_CAPACITY
        self.store = {}
        self.storage_left = CACHE_CAPACITY
        self.all_files = all_files

        for i in range(len(init_file_list)):
            if not self.__add_file__(init_file_list[i]):
                break

    def __add_file__(self,file_index):
        if file_index not in self.store:
            if self.storage_left > self.all_files.size[file_index]:
                self.store[file_index] = self.all_files.size[file_index]
                self.storage_left -= self.all_files.size[file_index]
            else:
                return False
        return True
            



class Simulator_Env():
    def __init__(self,
    cache_init_files=[]):

        self.sim_q = Simulation_Q()
        self.files = Files()
        self.cache = Cache(self.files,cache_init_files)
        self.fifo = []
        self.log = []


class Event:

    def __init__(self, sim: Simulator_Env, create_time: int, parent:object=None):
        self.sim = sim
        self.create_time = create_time
        self.process_time = create_time
        self.parent = parent
        self.name = 'Event'
        self.__enqueue__()

    def get_super_parent(self):
        node = self
        while(node.parent is not None):
            node = node.parent
        return node

    def __lt__(self,any):
        return True

    def __gt__(self,any):
        return True

    def __enqueue__(self):
        pass


    def process(self):
        pass

class E_new_req(Event):
    def __enqueue__(self):
        self.name = 'New Request'
        self.file_index = np.argmax(np.random.multinomial(1,self.sim.files.popularity))
        self.file_size = self.sim.files.size[self.file_index]        
        # self.process_time = self.create_time + (self.file_size/INSTITUTIONAL_BANDWIDTH)
        self.sim.sim_q.push([self.process_time,self])

    def process(self):
        if self.file_index in sim.cache.store:
            E_file_recieved(self.sim,self.process_time,self)
        else:
            E_arrive_at_queue(self.sim,self.process_time,self)
        


class E_get_new_reqs(Event):

    def __enqueue__(self):
        self.name = 'Get New Requests'
        self.sim.sim_q.push([self.process_time,self])

    def process(self):
        reqs_to_handle = np.random.poisson(POISSON_PROCESS_NEW_REQUESTS_LAMBDA)
        for i in range(int(reqs_to_handle)):
            E_new_req(self.sim, self.process_time)


class E_file_recieved(Event):
    def __enqueue__(self):
        self.name = 'File recieved'
        initial_req = self.get_super_parent()
        file_size = self.sim.files.size[initial_req.file_index]
        self.process_time = self.create_time + (file_size/ INSTITUTIONAL_BANDWIDTH)
        self.sim.sim_q.push([self.process_time,self])

    def process(self):
        initial_req = self.get_super_parent()
        self.sim.log.append(self.process_time - initial_req.create_time)


class E_arrive_at_queue(Event):
    def __enqueue__(self):
        self.name = 'Arrive at queue'
        self.process_time = self.create_time + np.random.lognormal(LOGNORMAL_PROCESS_ARRIVE_AT_QUEUE_MEAN,
                                                                    LOGNORMAL_PROCESS_ARRIVE_AT_QUEUE_SIGMA)
        
        self.sim.sim_q.push([self.process_time,self])

    def process(self):
        if len(self.sim.fifo):
            self.sim.fifo.append(self)
        else:
            E_depart_from_queue(self.sim,self.process_time,self)


class E_depart_from_queue(Event):
    def __enqueue__(self):
        self.name = 'Depart from queue'
        initial_req = self.get_super_parent()
        self.process_time = self.create_time + (initial_req.file_size/ACCESS_LINK_BANDWIDTH)
        self.sim.sim_q.push([self.process_time,self])

    def process(self):
        initial_req = self.get_super_parent()
        sim.cache.__add_file__(initial_req.file_index)
        E_file_recieved(self.sim,self.process_time,self)
        if len(sim.fifo):
            event = sim.fifo.pop(0)
            E_depart_from_queue(self.sim,self.process_time,event)











In [48]:
# initialize simulator environment
sim = Simulator_Env()

#initialize new req events to be processed at every second
for i in range(TOTAL_TIME_TO_RUN):
    E_get_new_reqs(sim,i)


#Main simulator loop
while(len(sim.sim_q.q)):

    e = sim.sim_q.pop()
    print(e.name,e.process_time)
    e.process()


Get New Requests 0
New Request 0
Arrive at queue 0.7092500673339114
Depart from queue 0.7163167445980967
File recieved 0.7173767461877245
Get New Requests 1
New Request 1
New Request 1
Arrive at queue 1.3543781792128813
Depart from queue 1.3550470886108752
File recieved 1.3551474250205744
Arrive at queue 1.5528180855799685
Depart from queue 1.5796826144980243
File recieved 1.5837122938357326
Get New Requests 2
New Request 2
New Request 2
File recieved 2.000100336409699
Get New Requests 3
Arrive at queue 3.543594734161434
Depart from queue 3.5928666305441133
File recieved 3.600257415001515
Get New Requests 4
New Request 4
New Request 4
Arrive at queue 4.23993750925309
Depart from queue 4.356490450898768
File recieved 4.373973392145619
Arrive at queue 4.630600261893644
Get New Requests 5
Depart from queue 5.619152220441045
File recieved 5.767435014223155
Get New Requests 6
Get New Requests 7
Get New Requests 8
Get New Requests 9


In [49]:
#initialize files
# Files - Array of 


sim.log

[0.7173767461877245,
 0.3551474250205744,
 0.5837122938357326,
 0.00010033640969897561,
 1.600257415001515,
 0.3739733921456194,
 1.767435014223155]